<a href="https://colab.research.google.com/github/Noor-Z1/Graduation-Project/blob/main/intra_subject_locomotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Pre processing


In [ ]:
# this cell is for mounting the drive to access the data

from google.colab import drive

# you can access the extracted dataset from here:
# https://drive.google.com/drive/folders/19juiVcKYgml1CUT0hJgWYWkHwghX2O5i?usp=sharing

# after having the dataset in your drive, mount google drive

drive.mount("/content/gdrive")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install scikeras

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# this cell is for preprocessing the sensors_data.txt
# to understand the columns of sensors data check: column_names.txt

import pandas as pd

# we choose 1 to 250 as the column_names.txt has defined the columns in this way
start_column = 1
end_column = 250


def preprocess_sensors_data(file_path):
    # Generate column names as a range of numbers
    column_names = list(range(start_column, end_column + 1))
    df = pd.read_csv(file_path, sep=' ')
    df.columns = column_names
    return df

def concatenate_sensor_data(base_path, ranges, drill=False):
  # Create an empty DataFrame with specified column names
    column_names = list(range(start_column, end_column + 1))
    df_concatenated = pd.DataFrame(columns=column_names)
    for range_tuple in ranges:
        for i in range_tuple[0]:
            for j in range_tuple[1]:
                file_path = f"{base_path}/S{i}-ADL{j}/S{i}-ADL{j}_sensors_data.txt"
                print(file_path)  # For demonstration, you can remove this line
                df = preprocess_sensors_data(file_path)  # Assuming preprocess_sensors_data is defined elsewhere
                df_concatenated = pd.concat([df_concatenated, df], ignore_index=True)
    if drill:
      for i in ranges[0][0]:
        # lastly include the drill run
        file_path = '/content/gdrive/MyDrive/Opportunity_extracted/data/S{}-Drill/S{}-Drill_sensors_data.txt'.format(i, i)
        print(file_path)
        df = preprocess_sensors_data(file_path)
        df_concatenated = pd.concat([df_concatenated, df])

    return df_concatenated


# ***Subject 1***

Reproducing the results of the benchmarked approach
"We train user-specific classifiers, always using as training set
the data of ADL1, ADL2, ADL3 and Drill sessions. We report classification performance for each subject on a testing set composed of
ADL4 and ADL5." https://www.sciencedirect.com/science/article/pii/S0167865512004205?ref=pdf_download&fr=RR-2&rr=86431f0ca9adc2d2

In [ ]:
# train set -> person 1 -> ADl 1, 2 and 3 and DRILL


# Define the ranges for different parts of the data
# here the first range will specify which range of subjects to look at
# and the second range will specify which range of ADLs for every subject to look at


ranges = [(range(1, 2), range(1, 4))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_train = concatenate_sensor_data(base_path, ranges, drill=True)


/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL1/S1-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL2/S1-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL3/S1-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-Drill/S1-Drill_sensors_data.txt


In [ ]:
# test set -> person 1, ADL 4 and 5

ranges = [(range(1, 2), range(4, 6))]

# Define the base path
base_path = '/content/gdrive/MyDrive/Opportunity_extracted/data'

# Call the function to concatenate the sensor data
df_test = concatenate_sensor_data(base_path, ranges, drill=False)

/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL4/S1-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S1-ADL5/S1-ADL5_sensors_data.txt


## *Subject 1 - Pre-Processing and EDA*

1.1 Repeating the previous value for NaN

In [ ]:
df_train

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,124.0,978.0,-389.0,-7.0,1014.0,199.0,124.0,968.0,123.0,...,5789.0,2908.0,1443.0,0,0,0,0,0,0,0
1,67,102.0,996.0,-440.0,-49.0,1024.0,193.0,127.0,1001.0,113.0,...,5789.0,2910.0,1440.0,0,0,0,0,0,0,0
2,100,59.0,861.0,-384.0,-9.0,1023.0,202.0,110.0,1007.0,106.0,...,5789.0,2912.0,1440.0,0,0,0,0,0,0,0
3,133,119.0,946.0,-426.0,-22.0,1026.0,188.0,98.0,1001.0,92.0,...,5791.0,2915.0,1442.0,0,0,0,0,0,0,0
4,167,99.0,972.0,-365.0,-3.0,1027.0,162.0,72.0,994.0,136.0,...,5790.0,2917.0,1445.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54960,1832015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3926.0,1613.0,869.0,0,0,0,0,0,0,0
54961,1832048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3073.0,1262.0,680.0,0,0,0,0,0,0,0
54962,1832082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2202.0,904.0,488.0,0,0,0,0,0,0,0
54963,1832115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1371.0,563.0,304.0,0,0,0,0,0,0,0


In [ ]:
# Fill NaN values with the previous non-NaN value
df_train_filled = df_train.fillna(method='ffill')

In [ ]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isnull().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(171575, 250)
35     0.295323
37     0.295323
36     0.295323
220    0.293347
221    0.293347
222    0.293347
dtype: float64


In [ ]:
print(df_train_filled.shape)
nan_stats = df_train_filled.isna().sum().sort_values(ascending = False)/df_train_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(171575, 250)
35     0.295323
37     0.295323
36     0.295323
220    0.293347
221    0.293347
222    0.293347
dtype: float64


even after filling with the previous value we can see the problematic columns - they never had enough 'previous' values to fill the NaN values with

In [ ]:
df_test

,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
0,33,-81.0,1057.0,10.0,-273.0,995.0,9.0,7.0,989.0,-9.0,...,5899.0,2748.0,1478.0,0,0,0,0,0,0,0
1,67,-92.0,1035.0,57.0,-294.0,985.0,0.0,26.0,998.0,-26.0,...,5900.0,2745.0,1476.0,0,0,0,0,0,0,0
2,100,-88.0,1032.0,92.0,-240.0,1014.0,32.0,7.0,995.0,-39.0,...,5902.0,2743.0,1475.0,0,0,0,0,0,0,0
3,133,-95.0,1027.0,87.0,-196.0,1002.0,38.0,38.0,1000.0,-96.0,...,5905.0,2740.0,1474.0,0,0,0,0,0,0,0
4,167,-99.0,1040.0,90.0,-225.0,992.0,30.0,71.0,1026.0,-55.0,...,5909.0,2738.0,1474.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63075,1004057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
63076,1004090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
63077,1004123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
63078,1004157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [ ]:
# Fill NaN values with the previous non-NaN value
df_test_filled = df_test.fillna(method='ffill')

In [ ]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isnull().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(63080, 250)
35    0.518025
36    0.518025
37    0.518025
dtype: float64


In [ ]:
print(df_test_filled.shape)
nan_stats = df_test_filled.isna().sum().sort_values(ascending = False)/df_test_filled.shape[0]

# print nan stats where the percentage is greater than 0.1 or 1%
print(nan_stats[nan_stats > 0.05])

(63080, 250)
35    0.518025
36    0.518025
37    0.518025
dtype: float64


ROBUST SCALER -> better with outliers
https://scikit-learn.org/0.18/auto_examples/preprocessing/plot_robust_scaling.html

a set of selected features

In [ ]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_filled[df_train_filled.columns[l]]
df_test_filtered = df_test_filled[df_test_filled.columns[l]]

In [ ]:
# dropping zero label
df_train_filtered = df_train_filtered[df_train_filtered[244] != 0]
df_test_filtered = df_test_filtered[df_test_filtered[244] != 0]

print(df_train_filtered[244].value_counts())
print(df_test_filtered[244].value_counts())

244
1    65643
2    43058
4    26909
5     3817
Name: count, dtype: int64
244
1    19742
4    13667
2    11664
5     2218
Name: count, dtype: int64


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Test Accuracy (KNN):", test_accuracy_knn)
print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))


Train Accuracy (KNN): 0.8111510791366906
Classification Report (Train):
              precision    recall  f1-score   support

           1       0.80      0.87      0.83       272
           2       0.79      0.68      0.73       163
           4       0.90      0.90      0.90       106
           5       0.67      0.53      0.59        15

    accuracy                           0.81       556
   macro avg       0.79      0.75      0.76       556
weighted avg       0.81      0.81      0.81       556

Test Accuracy (KNN): 0.6808510638297872
Classification Report (Test):
              precision    recall  f1-score   support

           1       0.57      0.86      0.69        71
           2       0.58      0.20      0.30        54
           4       0.92      0.91      0.92        54
           5       0.78      0.78      0.78         9

    accuracy                           0.68       188
   macro avg       0.71      0.69      0.67       188
weighted avg       0.68      0.68      0.65

In [ ]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = RobustScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=13,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.91
              precision    recall  f1-score   support

           1       0.93      0.88      0.91     19693
           2       0.83      0.90      0.86     12918
           4       1.00      0.98      0.99      8073
           5       1.00      1.00      1.00      1145

    accuracy                           0.91     41829
   macro avg       0.94      0.94      0.94     41829
weighted avg       0.91      0.91      0.91     41829



In [ ]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.93

Classification Report:
              precision    recall  f1-score   support

           1       0.93      0.90      0.92     19742
           2       0.85      0.89      0.87     11664
           4       0.98      1.00      0.99     13667
           5       1.00      1.00      1.00      2218

    accuracy                           0.93     47291
   macro avg       0.94      0.95      0.94     47291
weighted avg       0.93      0.93      0.93     47291



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Note that these results might not represent the generalization of the model so well since they are based on the same subject. Even if we do cross validation, the subset picked each time will be of the same person "that the samples produced by the same subjects are likely to be
correlated due to diverse factors. Hence, k-fold cross validation may
overestimate the performance of activity recognizers, in particular
when overlapping sliding windows are used. "
check paper: https://arxiv.org/pdf/1904.02666.pdf

In [ ]:
# lets try different cross validation now

In [ ]:
# change test set

import pandas as pd

# we choose 1 to 250 as the column_names.txt has defined the columns in this way
start_column = 1
end_column = 250

# Generate column names as a range of numbers
column_names = list(range(start_column, end_column + 1))

# Create an empty DataFrame with specified column names
df_test2 = pd.DataFrame(columns=column_names)

# change the file path accordingly
for i in range(2,3):
  for j in range(1,6):
    file_path = '/content/gdrive/MyDrive/Opportunity_extracted/data/S{}-ADL{}/S{}-ADL{}_sensors_data.txt'.format(i, j, i, j)
    print(file_path)
    df = preprocess_sensors_data(file_path)
    df_test2 = pd.concat([df_test2, df],ignore_index=True)

/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL1/S2-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL2/S2-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL3/S2-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL4/S2-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL5/S2-ADL5_sensors_data.txt


In [ ]:
df_test_interpolated = df_test2.fillna(df_test2.mean())

In [ ]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_test_filtered = df_test_interpolated[df_test_interpolated.columns[l]]

In [ ]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.64

Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.77      0.58     38599
           1       0.78      0.60      0.68     53010
           2       0.56      0.65      0.60     34912
           4       0.98      0.64      0.77     40345
           5       1.00      0.08      0.15      4914

    accuracy                           0.64    171780
   macro avg       0.76      0.55      0.56    171780
weighted avg       0.72      0.64      0.65    171780



In [ ]:
# train set -> person 2 -> ADl 1, 2 and 3 and DRILL


import pandas as pd

# we choose 1 to 250 as the column_names.txt has defined the columns in this way
start_column = 1
end_column = 250

# Generate column names as a range of numbers
column_names = list(range(start_column, end_column + 1))

# Create an empty DataFrame with specified column names
df_train = pd.DataFrame(columns=column_names)

# change the file path accordingly
for i in range(2,3):
  for j in range(1,4):
    file_path = '/content/gdrive/MyDrive/Opportunity_extracted/data/S{}-ADL{}/S{}-ADL{}_sensors_data.txt'.format(i, j, i, j)
    print(file_path)
    df = preprocess_sensors_data(file_path)
    df_train = pd.concat([df_train, df],ignore_index=True)


for i in range(2,3):
  # lastly include the drill run
  file_path = '/content/gdrive/MyDrive/Opportunity_extracted/data/S{}-Drill/S{}-Drill_sensors_data.txt'.format(i, i)
  print(file_path)
  df = preprocess_sensors_data(file_path)
  df_train = pd.concat([df_train, df])

/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL1/S2-ADL1_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL2/S2-ADL2_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL3/S2-ADL3_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-Drill/S2-Drill_sensors_data.txt


In [ ]:
# test set -> person 2, ADL 4 and 5

# we choose 1 to 250 as the column_names.txt has defined the columns in this way
start_column = 1
end_column = 250

# Generate column names as a range of numbers
column_names = list(range(start_column, end_column + 1))

# Create an empty DataFrame with specified column names
df_test = pd.DataFrame(columns=column_names)

# change the file path accordingly
for i in range(2,3):
  for j in range(4,6):
    file_path = '/content/gdrive/MyDrive/Opportunity_extracted/data/S{}-ADL{}/S{}-ADL{}_sensors_data.txt'.format(i, j, i, j)
    print(file_path)
    df = preprocess_sensors_data(file_path)
    df_test = pd.concat([df_test, df],ignore_index=True)

/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL4/S2-ADL4_sensors_data.txt
/content/gdrive/MyDrive/Opportunity_extracted/data/S2-ADL5/S2-ADL5_sensors_data.txt


In [ ]:
df_train_interpolated = df_train.fillna(df_train.mean())

In [ ]:
df_test_interpolated = df_test.fillna(df_test.mean())

In [ ]:
l = [4,5,6, 7,8,9, 10,11,12, 13,14,15, 16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,37,38,39,40,56,84,85,97,102,106,107,127,129,243]
df_train_filtered = df_train_interpolated[df_train_interpolated.columns[l]]
df_test_filtered = df_test_interpolated[df_test_interpolated.columns[l]]

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Extract train and test data
train_data = df_train_filtered.drop(columns=[244])
test_data = df_test_filtered.drop(columns=[244])

# Extract train and test labels
train_labels = df_train_filtered[244].values
test_labels = df_test_filtered[244].values


def extract_features_and_labels(data, labels):
    features = []
    feature_labels = []
    num_samples = len(data)

    # Define window length and step size (in number of samples)
    window_length = 500  # 500 samples
    step_size = 250      # 250 samples

    # Iterate over the data with the specified step size
    for start in range(0, num_samples - window_length + 1, step_size):
        end = start + window_length
        window_data = data.iloc[start:end, :]  # Get data within the window for all sensors

        # Calculate the mean value of the sensor data within the window for each sensor
        mean_values = window_data.mean(axis=0)

        # Append the mean values to the list of features
        features.append(mean_values)

        # Determine the label for this window
        window_label = labels[start]  # Assuming the label corresponds to the start of the window
        feature_labels.append(window_label)

    return features, feature_labels


# Extract features and labels from train_data
train_features, train_feature_labels = extract_features_and_labels(train_data, train_labels)

# Extract features from test_data (assuming you also want to keep track of test labels)
test_features, test_feature_labels = extract_features_and_labels(test_data, test_labels)

# Normalize features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# # Train a Random Forest classifier
# rf_model = RandomForestClassifier(n_estimators=200, max_depth=8, max_features='sqrt', class_weight='balanced')
# rf_model.fit(train_features_normalized, train_feature_labels)

# # Evaluate the model
# train_predictions = rf_model.predict(train_features_normalized)
# test_predictions = rf_model.predict(test_features_normalized)

# train_accuracy = accuracy_score(train_feature_labels, train_predictions)
# test_accuracy = accuracy_score(test_feature_labels, test_predictions)

# print("Train Accuracy:", train_accuracy)
# print("Test Accuracy:", test_accuracy)

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # You can adjust n_neighbors as needed
knn_model.fit(train_features_normalized, train_feature_labels)

# Evaluate the model
train_predictions_knn = knn_model.predict(train_features_normalized)
test_predictions_knn = knn_model.predict(test_features_normalized)

train_accuracy_knn = accuracy_score(train_feature_labels, train_predictions_knn)
test_accuracy_knn = accuracy_score(test_feature_labels, test_predictions_knn)

print("Train Accuracy (KNN):", train_accuracy_knn)
print("Test Accuracy (KNN):", test_accuracy_knn)

# Generate classification report
print("Classification Report (Train):")
print(classification_report(train_feature_labels, train_predictions_knn))

print("Classification Report (Test):")
print(classification_report(test_feature_labels, test_predictions_knn))


Train Accuracy (KNN): 0.8190327613104524
Test Accuracy (KNN): 0.6848249027237354
Classification Report (Train):
              precision    recall  f1-score   support

           0       0.77      0.95      0.85       111
           1       0.83      0.85      0.84       250
           2       0.74      0.66      0.70       149
           4       0.95      0.89      0.92       119
           5       0.67      0.17      0.27        12

    accuracy                           0.82       641
   macro avg       0.79      0.70      0.72       641
weighted avg       0.82      0.82      0.81       641

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.74      0.79      0.76        63
           1       0.52      0.73      0.61        67
           2       0.53      0.28      0.36        58
           4       0.97      0.93      0.95        61
           5       0.67      0.50      0.57         8

    accuracy                           0.68

In [ ]:
X = np.array(df_train_filtered.drop([244], axis=1))
y = np.array(df_train_filtered[244])
y = y.astype('int')

scaler = MinMaxScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest


# Create a RandomForestClassifier with class weights
rf_model = RandomForestClassifier(class_weight='balanced',max_depth=14,max_features='sqrt',n_estimators=400)

rf_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with selected features: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy with selected features: 0.90
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      8594
           1       0.93      0.87      0.90     18497
           2       0.81      0.90      0.85     11523
           4       0.99      0.97      0.98      8663
           5       0.89      0.98      0.93       905

    accuracy                           0.90     48182
   macro avg       0.91      0.92      0.91     48182
weighted avg       0.91      0.90      0.90     48182



In [ ]:
X = np.array(df_test_filtered.drop([244], axis=1))
y = np.array(df_test_filtered[244])
y = y.astype('int')

X_scaled = scaler.transform(X)
y_pred = rf_model.predict(X_scaled)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y, y_pred))

Accuracy: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.69      0.80     16086
           1       0.80      0.85      0.83     17844
           2       0.70      0.89      0.78     13697
           4       0.99      0.98      0.99     15047
           5       0.94      0.95      0.94      1898

    accuracy                           0.85     64572
   macro avg       0.88      0.87      0.87     64572
weighted avg       0.87      0.85      0.85     64572

